In [95]:
import pandas as pd
import re

# 데이터 로딩
cnn = pd.read_csv("C:/Users/ad644/Downloads/model_result.csv")
sur = pd.read_csv("C:/Users/ad644/Downloads/survey_result.csv")

In [96]:
a = cnn
b = sur

In [103]:
a

,이미지,모험적인,자연/힐링,예술적인,문화적인,전통적인,즐거운/활발한
0,"C:/Users/ad644/OneDrive/바탕 화면/레포츠_육상/서울/야영장, 오...",85.59,28.47,14.29,13.84,11.86,1.68


In [90]:
b

,분류,모험적인,자연/힐링,예술적인,문화적인,전통적인,즐거운/활발한
0,육상레포츠 ATV,26,2,0,0,0,12
1,육상레포츠 야영,0,37,0,0,0,3
2,문화시설 전시관,0,0,33,7,0,0
3,역사관광지 절/궁궐,0,1,1,3,35,0
4,자연관광지 바다,0,39,0,0,0,1
5,수상레포츠 레프팅,10,0,0,0,0,30
6,수상레포츠 민물낚시,2,30,1,1,0,6
7,문화시설 박물관,0,0,2,11,26,1
8,문화시설 외국문화원,0,0,12,20,8,0
9,복합레포츠,24,8,0,0,0,8


In [91]:
def cnn_f(data):
    
    top_3_cols = data.drop('이미지', axis=1).apply(lambda x: x.nlargest(3).index.tolist(), axis=1)
    data = data[['이미지'] + top_3_cols.iloc[0]]
    data.set_index('이미지',  drop = True)
    w_mapping = {list(data.items())[1][1][0]: 2, list(data.items())[2][1][0]: 3, list(data.items())[3][1][0]: 5}


    data = {key: [float(re.search(r'\d+\.\d+', str(value[0])).group()) * w_mapping.get(value[0], 1)] if key != '이미지' else value for key, value in data.items()}

    return data

In [92]:
def sur_f(data):
    
    data = data[data['분류'] == '육상레포츠 야영']
    data.set_index('분류', drop = True)
    top_3_cols = data.drop('분류', axis=1).apply(lambda x: x.nlargest(3).index.tolist(), axis=1)
    data = data[['분류'] + top_3_cols.iloc[0]]
    data = data.to_dict(orient='list')

    w_mapping = {list(data.items())[1][1][0]: 2, list(data.items())[2][1][0]: 3, list(data.items())[3][1][0]: 5}
    
    for key, value in data.items():
        if key != '이미지':
            match = re.search(r'\d+\.\d+', str(value[0]))
            if match:
                data[key] = [float(match.group()) * w_mapping.get(value[0], 1)]
            else:
                # Handle the case when no match is found
                data[key] = data[key]
   

    return data

In [113]:
final_result = pd.DataFrame(columns = ['이미지', '분위기'])

def final(c, d):

    if list(c.items())[1][1][0] > 80:
        final_keyword = list(c.keys())[1]

        final_result['이미지'] = c['이미지']
        
        final_result['분위기'] = final_keyword
    
    else:
        cnn_eee = cnn_f(c)
        sur_eee = sur_f(d)
    
        result_dict = {}

        w1 = 3
        w2 = 2

        # 두 딕셔너리에서 공통된 키 찾기
        common_keys = set(cnn_eee.keys()) & set(sur_eee.keys())

        # 각 키에 대해 값을 더해서 새로운 딕셔너리 만들기
        for key in common_keys:
            # 값이 리스트인지 확인
            if isinstance(cnn_eee[key], list) and isinstance(sur_eee[key], list):
                # 각 리스트의 원소에 가중치를 곱하고 결과를 합산
                result_dict[key] = sum(x * w1 + y * w2 for x, y in zip(cnn_eee[key], sur_eee[key]))
            else:
                # 리스트가 아닌 경우에는 값에 직접 가중치를 곱함
                result_dict[key] = cnn_eee[key] * w1 + sur_eee[key] * w2

        max_value = max(result_dict.values())
        filtered_result = {key: value for key, value in result_dict.items() if value == max_value}
        final_key_list = list(filtered_result.keys())
        final_keyword = ''.join(final_key_list)

        final_result['이미지'] = c['이미지']
        
        final_result['분위기'] = final_keyword
        
    return final_result

In [105]:
cnn_f(a)

{'이미지': 0    C:/Users/ad644/OneDrive/바탕 화면/레포츠_육상/서울/야영장, 오...
 Name: 이미지, dtype: object,
 '모험적인': [171.18],
 '자연/힐링': [85.41],
 '예술적인': [71.44999999999999]}

In [106]:
sur_f(b)

{'분류': ['육상레포츠 야영'], '자연/힐링': [37], '즐거운/활발한': [3], '모험적인': [0]}

In [114]:
final(a, b)

,이미지,분위기
0,"C:/Users/ad644/OneDrive/바탕 화면/레포츠_육상/서울/야영장, 오...",모험적인
